In [1]:
source("Main.R")
source("Conf.R")
source("Utilities.R")
source("DAVID_query.R")
library("factoextra")
library("maptree")
library(repr)


corrplot 0.84 loaded


********************************************************

Note: As of version 1.0.0, cowplot does not change the

  default ggplot2 theme anymore. To recover the previous

  behavior, execute:
  theme_set(theme_cowplot())

********************************************************



Attaching package: ‘pls’


The following object is masked from ‘package:corrplot’:

    corrplot


The following object is masked from ‘package:stats’:

    loadings


Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following o

In [2]:
proComp <- data.frame(fread("/home/beraslan/jovian-work/analysisSingle/R/allComplexes.txt"), stringsAsFactors = F)
proComp <- proComp[proComp$Organism == "Mouse",]

In [3]:
ProteinComplexes=list()

for(i in 1:nrow(proComp)){
  ProteinComplexes <- lappend(ProteinComplexes, strsplit(as.character(proComp[i,"subunits.Gene.name."]) , ";")[[1]])
}
names(ProteinComplexes) <- proComp$ComplexName
ProteinComplexes <- ProteinComplexes[!duplicated(names(ProteinComplexes))]


In [4]:
testSignOfKOOnProComp <- function(KOEffectedGenes, ProCompGenes, Universe, KOGene, ProComplex, allTests){
    
    if(length(ProCompGenes[ProCompGenes %in% KOEffectedGenes]) > 1){
        myDF = data.frame(koGenes = c(length(KOEffectedGenes[KOEffectedGenes %in% ProCompGenes]), length(ProCompGenes[ProCompGenes %ni% KOEffectedGenes])),
              restNo = c( length(KOEffectedGenes[KOEffectedGenes %ni% ProCompGenes]), length(Universe[Universe %ni% unique(c(KOEffectedGenes, ProCompGenes))])))

        testRes_g <- fisher.test(x= myDF, alternative = "greater")
        return(c(KOGene, ProComplex, testRes_g$p.value))
    }else{
        return(c(KOGene, ProComplex, 1))
    }

}

In [5]:
coefsAll <- read.csv("/home/beraslan/jovian-work/analysisSingle/MixedEffectLMOutputs/ME_LMBetaCoefsALL.csv", row.names = 1)
pValsAll <- read.csv("/home/beraslan/jovian-work/analysisSingle/MixedEffectLMOutputs/ME_LMPValuesALL.csv", row.names = 1)
pValsAllAdj <- sapply(pValsAll, function(x){p.adjust(x, method = "fdr", n = length(x))})
rownames(pValsAllAdj) <- rownames(pValsAll)
coefsAll[pValsAllAdj > 0.1] = 0

In [6]:
coefsAllSign <-  read.csv("/home/beraslan/jovian-work/analysisSingle/MixedEffectLMOutputs/ME_SignificantBetaCoefs.csv", row.names = 1)

In [7]:
coefsAll <- coefsAll[rownames(coefsAllSign),]

In [8]:
dim(coefsAll)

[1]  329 6685

In [9]:
k <- apply(coefsAll, 1, function(x){colnames(coefsAll)[which(x != 0)]})


In [10]:
names(k) <- sapply(names(k), function(x){strsplit(x,"_")[[1]][2]})

In [14]:
coefsAll["GENE_Rack1_", k[["Rack1"]]]

,Rpl7,Cox5b,Rpl31,Rnf149,Eef1b2,Rpl37a,Ptma,St8sia4,Serpinb2,Dbi,⋯,Fau,Ppp1r14b,Cox8a,Eef1g,Fth1,Cd274,Pten,Scd2,Usmg5,Shoc2
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
GENE_Rack1_,-0.397,-0.381,-0.247,0.259,-0.291,-0.487,-0.241,0.229,0.261,-0.253,⋯,-0.29,-0.261,-0.418,-0.359,0.138,0.248,0.255,-0.242,-0.296,0.256


In [ ]:
allTests = list()
for(i in 1:length(k)){
    for(j in 1:length(ProteinComplexes)){
        allTests <- lappend(allTests, testSignOfKOOnProComp(KOEffectedGenes=k[[i]],
                                          ProCompGenes=ProteinComplexes[[j]],
                                          Universe=colnames(coefsAll),
                                          KOGene=names(k)[i],
                                          ProComplex=names(ProteinComplexes)[j],
                                          allTests = allTests))
    }
}    

In [ ]:
allTests <- data.frame(do.call(rbind, allTests), stringsAsFactors = FALSE)

In [ ]:
colnames(allTests) = c("KOGene", "ProComplex", "Pval")

In [ ]:
allTests <- allTests[allTests$Pval < 0.1,]

In [ ]:
length(unique(allTests$ProComplex))

In [ ]:
dim(allTests)

In [ ]:
allTestsSplit <- split(allTests, f = allTests$ProComplex)

In [ ]:
myRes <- sapply(allTestsSplit, function(x){return(paste(unique(x[,"KOGene"]), collapse = ", "))})

In [ ]:
myRes

In [ ]:
myRes

In [ ]:
head(allTests)

In [ ]:
myResMat = matrix(0, ncol = length(unique(allTests$KOGene)), nrow = length(unique(allTests$ProComplex)))

In [ ]:
dim(myResMat)

In [ ]:
colnames(myResMat) = unique(allTests$KOGene)
rownames(myResMat) = unique(allTests$ProComplex)

In [ ]:
myResMat = data.frame(myResMat)

In [ ]:
for( i in 1:nrow(allTests)){
    myResMat[allTests[i,"ProComplex"], allTests[i,"KOGene"]] = 1
}

In [ ]:
guideModulesN <- data.frame(read.csv("/home/beraslan/jovian-work/analysisSingle/ME_GuideModules_leiden_6_Modules.csv"),  row.names = 1)
rownames(guideModulesN) <- guideModulesN$GuideName 
guideModulesN$GuideName <- NULL
guideModulesN$GuideGroup <- factor(guideModulesN$GuideGroup)

In [ ]:
guideModulesN$GuideGroup <- paste0("KOGroup_", guideModulesN$GuideGroup)
head(guideModulesN)

In [ ]:
guideModulesN$GuideColor <- NULL

In [ ]:
head(guideModulesN)

In [ ]:
annoCols<-list( GuideGroup=c(KOGroup_0="#1f77b4", KOGroup_1="#ff7f0e", KOGroup_2="#279e68", KOGroup_3="#d62728", KOGroup_4="#aa40fc", KOGroup_5="#8c564b"))
  

In [ ]:
head(myResMat)

In [ ]:
options(repr.plot.width=25, repr.plot.height=10)
pheatmap(myResMat, clustering_method="ward.D2", treeheight_col = 0, treeheight_row = 0, color = c("lightgrey", "red"),  annotation_colors = annoCols, annotation_col = guideModulesN)

In [ ]:
KOEffectedGenes[KOEffectedGenes %ni% ProCompGenes]

In [ ]:
length(ProCompGenes[ProCompGenes %ni% KOEffectedGenes])

In [ ]:
unique(c(KOEffectedGenes, ProCompGenes))

In [ ]:
length(universe[universe %ni% unique(c(KOEffectedGenes, ProCompGenes))])